In [1]:
denoise_tool_file = '/Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/dada2-denoise-single.cwl'
visualize_tool_file = '/Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/metadata-tabulate.cwl'

In [7]:
from cwlgen.import_cwl import parse_cwl
denoise_tool = parse_cwl(denoise_tool_file)
visualize_tool = parse_cwl(visualize_tool_file)

In [9]:
def print_inputs_outputs(tool):
    print('inputs:')
    for inp in tool.inputs:
        print('\t', inp.id, '(', inp.type, ')')
    print('outputs:')
    for out in tool.outputs:
        print('\t', out.id, '(', out.type, ')')

In [11]:
print_inputs_outputs(denoise_tool)
print_inputs_outputs(visualize_tool)

inputs:
	 demultiplexed_seqs ( File )
	 trim_left ( int )
	 trunc_len ( int )
	 representative_sequences_filename ( string )
	 table_filename ( string )
	 denoising_stats_filename ( string )
outputs:
	 representative_sequences ( File )
	 table ( File )
	 denoising_stats ( File )
inputs:
	 input_file ( File )
	 visualization_filename ( string )
outputs:
	 visualization_artifact ( File )


In [ ]:
# Use case: Connect two tools as steps in a workflow and write out the workflow

In [39]:
import cwlgen
from cwlgen.workflow import Workflow, InputParameter
dada2_workflow = Workflow()

denoise_step = dada2_workflow.add('denoise', denoise_tool, {
    'demultiplexed_seqs': cwlgen.File('demultiplexed_seqs'),
    'trim_left': InputParameter('trim_left', param_type='int'),
})

denoise_step['denoising_stats'].store()

dada2_workflow.export()

#!/usr/bin/env cwl-runner

class: Workflow
cwlVersion: v1.0
inputs:
  demultiplexed_seqs: {id: demultiplexed_seqs, type: File}
outputs:
  denoise_denoising_stats: {id: denoise_denoising_stats, outputSource: denoise/denoising_stats,
    type: File}
steps:
  denoise:
    id: denoise
    in: {demultiplexed_seqs: demultiplexed_seqs}
    out: [representative_sequences, table, denoising_stats]
    run: /Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/dada2-denoise-single.cwl



In [27]:
dada2_workflow.export()

#!/usr/bin/env cwl-runner

class: Workflow
cwlVersion: v1.0
inputs:
  demultiplexed_seqs: {id: demultiplexed_seqs, type: File}
outputs: {}
steps:
  denoise:
    id: denoise
    out: [representative_sequences, table, denoising_stats]
    run: /Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/dada2-denoise-single.cwl



In [64]:
# Now instead trying the qiime2-01-import-data.cwl
from cwlgen.import_cwl import parse_cwl
from cwlgen import File, Workflow
from cwlgen.workflow import InputParameter

mkdir_file = '/Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/EMPSingleEndSequences-directory.cwl'
import_file = '/Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/tools-import.cwl'

mkdir_tool = parse_cwl(mkdir_file)
import_tool = parse_cwl(import_file)


w = Workflow()
sequences = File('sequences')
barcodes = File('barcodes')

# Add steps
step1 = w.add('make_import_directory', mkdir_tool, {
    'sequences': sequences, 
    'barcodes': barcodes,
})
step2 = w.add('make_sequences_artifact', import_tool, {
    'input_path': step1['dir'], 
    'type': InputParameter('type', param_type='int'),
})
step2['sequences_artifact'].store()

w.export()


#!/usr/bin/env cwl-runner

class: Workflow
cwlVersion: v1.0
inputs:
  barcodes: {id: barcodes, type: File}
  sequences: {id: sequences, type: File}
outputs:
  make_sequences_artifact_sequences_artifact: {id: make_sequences_artifact_sequences_artifact,
    outputSource: make_sequences_artifact/sequences_artifact, type: File}
steps:
  make_import_directory:
    id: make_import_directory
    in: {barcodes: barcodes, sequences: sequences}
    out: [dir]
    run: /Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/EMPSingleEndSequences-directory.cwl
  make_sequences_artifact:
    id: make_sequences_artifact
    in: {input_path: make_import_directory/dir}
    out: [sequences_artifact]
    run: /Users/dcl9/Code/yaml/bespin-cwl/tools/qiime2/tools-import.cwl

